[Reference](https://medium.com/voxel51/vggt-is-a-pure-neural-approach-to-3d-vision-32841d5e3c32)

In [1]:
pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.0/310.0 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.3/943.3 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.2 MB/s eta 

In [2]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub
import fiftyone.zoo as foz

dataset = load_from_hub("Voxel51/Total-Text-Dataset")

foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/vggt",
    overwrite=True
)

In [3]:
model = foz.load_zoo_model(
    "facebook/VGGT-1B",
    install_requirements=True,
    mode="crop", # you can also pass "pad",
    confidence_threshold=0.7
    )

# Apply to your dataset
dataset.apply_model(model, "depth_map_path")

In [4]:
import os
from pathlib import Path
import fiftyone as fo

grouped_dataset = fo.Dataset("vggt_results", overwrite=True)
grouped_dataset.add_group_field("group", default="rgb")

samples = []
for filepath in dataset.values("filepath"):
    path = Path(filepath)
    base_dir = path.parent
    base_name = path.stem

    # Create paths for each modality
    rgb_path = filepath
    depth_path = os.path.join(base_dir, f"{base_name}_depth.png")
    threed_path = os.path.join(base_dir, f"{base_name}.fo3d")

    group = fo.Group()
    samples.extend([
        fo.Sample(filepath=rgb_path, group=group.element("rgb")),
        fo.Sample(filepath=depth_path, group=group.element("depth")),
        fo.Sample(filepath=threed_path, group=group.element("threed"))
    ])

grouped_dataset.add_samples(samples)
fo.launch_app(grouped_dataset)  # View results interactively